In [32]:
import sys
sys.path.append('/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/')
from trained_untrained_results_funcs import find_best_layer,load_mean_sem_perf, loop_through_datasets
import numpy as np
from matplotlib import pyplot as plt

In [28]:
gpt2_xl_mp = np.load('/data/LLMs/data_processed/blank/acts/X_gpt2-xl-mp.npz')
data_labels = np.load('/data/LLMs/data_processed/blank/dataset/data_labels_blank.npy')


In [25]:
def shift_model_activations(model_acts, story_labels):

    samples = model_acts.shape[0]
    features = model_acts.shape[1]

    shifted_versions = np.zeros((samples, features*4))
    
    zero_pad = np.zeros(features)
        
    stories = np.unique(story_labels)
    for story in stories:
        story_idxs = np.argwhere(story_labels==story)
            
        model_acts_story = model_acts[story_idxs].squeeze()
        model_acts_story_2 = np.vstack((model_acts_story[1:, :], zero_pad)) # 2 second lag
        model_acts_story_6 = np.vstack((zero_pad, model_acts_story[:-1, :])) # 6 second lag
        model_acts_story_8 = np.vstack((zero_pad, zero_pad, model_acts_story[:-2, :])) # 8 second lag
        
        model_acts_time_lagged_story = np.hstack((model_acts_story_2, model_acts_story, model_acts_story_6, model_acts_story_8))
        
        shifted_versions[story_idxs.squeeze()] = model_acts_time_lagged_story
        
    return shifted_versions
    

In [29]:
gpt2_xl_mp_stacked = {}
for layer_name, layer in gpt2_xl_mp.items():
    
    gpt2_xl_mp_stacked[layer_name] = shift_model_activations(layer, data_labels)
    

In [30]:
np.savez("/data/LLMs/data_processed/blank/acts/X_timelagged-gpt2-xl-mp.npz", **gpt2_xl_mp_stacked)

In [33]:
subjects_storage = {}
subjects_storage['blank'] = np.load(f"'/data/LLMs/data_processed/blank/dataset/subjects.npy", allow_pickle=True)

layer_perf_dict, best_layer_blank, layer_perf_best  = find_best_layer(np.arange(49), '', '', resultsPath='/data/LLMs/brainscore/results_blank/', 
                    subjects=subjects_storage['blank'],
                    dataset='blank', perf='pearson_r', 
                    selected_network_indices = None, feature_extraction = '-mp', model_name='timelagged-gpt2-xl', seed_number=None, 
                    return_SE=False, niter=1)

FileNotFoundError: [Errno 2] No such file or directory: '/data/LLMs/data_processed/blank/subjects.npy'